In [1]:
const fs = require('fs');
const puppeteer = require('puppeteer');
const randomUseragent = require('random-useragent');
const StealthPlugin = require('puppeteer-extra-plugin-stealth');
const puppeteerExtra = require('puppeteer-extra');
puppeteerExtra.use(StealthPlugin());

<ref *1> PuppeteerExtra {
  _pptr: {
    connect: [Function: bound bound connect],
    createBrowserFetcher: [Function: bound createBrowserFetcher],
    defaultArgs: [Function: bound defaultArgs],
    executablePath: [Function: bound executablePath],
    launch: [Function: bound launch],
    WEB_PERMISSION_TO_PROTOCOL_PERMISSION: [Getter],
    Browser: [Getter],
    BrowserContext: [Getter],
    Page: [Getter],
    supportedMetrics: [Getter],
    unitToPixels: [Getter],
    JSHandle: [Getter],
    ElementHandle: [Getter],
    HTTPResponse: [Getter],
    DEFAULT_INTERCEPT_RESOLUTION_PRIORITY: [Getter],
    HTTPRequest: [Getter],
    InterceptResolutionAction: [Getter],
    headersArray: [Getter],
    STATUS_TEXTS: [Getter],
    Accessibility: [Getter],
    ARIAQueryHandler: [Getter],
    CDPBrowser: [Getter],
    CDPBrowserContext: [Getter],
    _connectToCDPBrowser: [Getter],
    BrowserWebSocketTransport: [Getter],
    ChromeTargetManager: [Getter],
    ConnectionEmittedEvents: [Gette

In [7]:
function wait(ms) {
  return new Promise((resolve) => setTimeout(resolve, ms));
}


In [2]:
async function searchTableValue(page, keyword, caption) {
  try {
    const value = await page.evaluate((word, cap) => {
      const captionElements = Array.from(document.querySelectorAll('caption'));
      for (const captionElement of captionElements) {
        const captionText = captionElement.innerText.trim();
        if (captionText === cap) {
          const tableElement = captionElement.parentElement;
          const thElements = Array.from(tableElement.querySelectorAll('.table-horizontal th'));
          for (const thElement of thElements) {
            const thText = thElement.innerText.trim();
            if (thText.includes(word)) {
              const tdElement = thElement.nextElementSibling;
              const tdText = tdElement.innerText.trim();
              return tdText.toString(); // Convert the value to a string
            }
          }
        }
      }
      return NaN.toString(); // Return NaN as a string
    }, keyword, caption, { timeout: 5000 });

    return value;
  } catch (error) {
    console.error(`Error occurred while searching for ${keyword}:`, error);
    return NaN.toString(); // Return NaN as a string in case of error
  }
}

In [3]:
async function openUrl(url) {
  try {
    const browser = await puppeteerExtra.launch({
      headless: true,
      args: [
        "--no-sandbox",
        "--disable-setuid-sandbox",
        "--disable-infobars",
        "--disable-dev-shm-usage",
        "--disable-browser-side-navigation",
        "--disable-features=site-per-process",
        "--disable-web-security",
      ],
    });
    const page = await browser.newPage();

    const userAgent = randomUseragent.getRandom();
    await page.setUserAgent(userAgent);

    const { width, height } = await page.evaluate(() => ({
      width: window.screen.width,
      height: window.screen.height,
    }));
    await page.setViewport({ width, height });

    await page.goto(url);

    return page;
  } catch (error) {
    console.error(error);
    throw error;
  }
}


In [4]:
async function extractUrlsFromPages(initialUrl, totalPages, urlFormat) {
  const urls = [];

  const browser = await puppeteer.launch();
  const page = await browser.newPage();

  for (let currentPage = 1; currentPage <= totalPages; currentPage++) {
    const url = urlFormat.replace("{page}", currentPage);

    await page.goto(url);

    const links = await page.$$eval(".item-container a.item-img", (elements) =>
      elements.map((element) => element.href.trim())
    );

    urls.push(...links);
    console.log(`Found ${links.length} links on page ${currentPage}`);
    await wait(3000);
  }

  await browser.close();

  return urls;
}

In [5]:
async function run() {
  // Define pagenow before using it
  const initialUrl1 = "https://www.newegg.com/Business-Laptops/SubCategory/ID-3413/Page-{page}?Tid=167751";
  const totalPages1 = 40;
  const urlFormat1 = initialUrl1;
  const urls1 = await extractUrlsFromPages(initialUrl1, totalPages1, urlFormat1);

  // Define pagenow2 before using it
  const initialUrl2 = "https://www.newegg.com/Laptops-Notebooks/SubCategory/ID-32/Page-{page}?Tid=6770";
  const totalPages2 = 100;
  const urlFormat2 = initialUrl2;
  const urls2 = await extractUrlsFromPages(initialUrl2, totalPages2, urlFormat2);

  const urls = [...urls1, ...urls2];

  return urls; // Return the obtained URLs
}

In [6]:
async function scrapePrices() {
  try {
    const urls = await run();
    console.log(urls);
    const laptopsData = [];

    const browser = await puppeteer.launch();
    const page = await browser.newPage();
    let loopCount = 0;
    for (const [index, url] of urls.entries()) {
      loopCount++; // Increment the loop count
      console.log(`Loop count: ${loopCount}`);
      let retryCount = 0;
      while (retryCount < 5) {
        try {
          await page.goto(url);
          await page.waitForSelector('.product-view-brand.has-brand-store .logo', { timeout: 5000 });

          const condition = await page.evaluate(() => {
            const conditionElement = document.querySelector('.product-offer .link-more');
            if (conditionElement) {
              const conditionText = conditionElement.innerText;
              if (conditionText.toLowerCase().includes('new')) {
                return 1;
              }
            }
            return 0;
          });

          console.log(`Condition: ${condition}`);

          const brand = await page.evaluate(() => {
            const brandElement = document.querySelector('.product-view-brand.has-brand-store .logo');
            if (brandElement) {
              return brandElement.getAttribute('alt');
            }
            return null;
          });

          const price = await page.evaluate(() => {
            const priceElement = document.querySelector('.product-price .price-current');
            if (priceElement) {
              const priceText = priceElement.innerText;
              return priceText;
            }
            return null;
          });

          console.log(`Price: ${price}`);

          const isButtonClicked = await page.evaluate(() => {
            const button = document.querySelector('.tab-nav.active');
            if (button) {
              button.click();
              return true;
            }
            return false;
          });

          if (isButtonClicked) {
            console.log('The button was clicked successfully.', url);
            await wait(5000);
          } else {
            console.log('Failed to click the button or expected changes did not occur for URL:', url);
          }

          const Memory = await searchTableValue(page, 'Memory', 'Memory');
          console.log(`Value: ${Memory}`);

          const cpu_speed = await searchTableValue(page, 'CPU Speed', 'CPU');
          console.log(`Value: ${cpu_speed}`);

          const Number_of_Cores = await searchTableValue(page, ' Number of Cores', 'CPU');
          const extractedNumber = Number_of_Cores && Number_of_Cores.length > 0 ? Number_of_Cores[0].match(/\d+/)?.[0] : null;
          console.log(`Value3: ${extractedNumber}`);

          const sizescreen = await searchTableValue(page, 'Screen Size', 'Display');
          console.log(`Value: ${sizescreen}`);

          const year = await searchTableValue(page, 'Date First Available', 'Additional Information');
          const cleanedYear = year ? year.match(/\d{4}$/)?.[0] : null;
          console.log(`Value: ${cleanedYear}`);

          const Resolution = await searchTableValue(page, 'Resolution', 'Display');
          console.log(`Value: ${Resolution}`);

          const ssd = await searchTableValue(page, 'SSD', 'Storage');
          console.log(`Value: ${ssd}`);

          const touchscreen = await searchTableValue(page, 'Touchscreen ', 'Display');
          const touchscreen_value = touchscreen === 'No' ? 0 : 1;
          console.log(`Value: ${touchscreen_value}`);

          const Webcam = await searchTableValue(page, 'Webcam', 'Supplemental Drive');
          const Webcam_value = Webcam === 'No' ? 0 : 1;
          console.log(`Value1: ${Webcam_value}`);

          const microsd = await searchTableValue(page, 'Micro SD', 'Storage');
          const microsd_value = microsd === 'No' ? 0 : 1;
          console.log(`Value2: ${microsd_value}`);

          const dedicated_graphics = await searchTableValue(page, 'Dedicated Graphics', 'Graphics Card');
          const dedicated_graphics_value = dedicated_graphics === 'No' ? 0 : 1;
          console.log(`Value: ${dedicated_graphics_value}`);

          const usb_connections = await searchTableValue(page, 'USB', 'USB Connections of Different Types');
          const usb_connections_value = usb_connections ? usb_connections.length : 0;
          console.log(`USB connections of different types: ${usb_connections_value}`);

          const total_usb_connections = await searchTableValue(page, 'Total USB Connections', 'USB Connections of Different Types');
          const total_usb_connections_value = total_usb_connections ? parseInt(total_usb_connections[0]) : 0;
          console.log(`Total USB connections: ${total_usb_connections_value}`);

          laptopsData.push({
            url,
            condition,
            brand,
            price,
            Memory,
            cpu_speed,
            Number_of_Cores: extractedNumber,
            sizescreen,
            year: cleanedYear,
            Resolution,
            ssd,
            touchscreen: touchscreen_value,
            Webcam: Webcam_value,
            microsd: microsd_value,
            dedicated_graphics: dedicated_graphics_value,
            usb_connections: usb_connections_value,
            total_usb_connections: total_usb_connections_value
          });

          break; // Exit the retry loop if the page opened successfully
        } catch (error) {
          console.error(`Failed to open URL: ${url}`);
          console.error(error);
          retryCount++; // Increment the retry count
          await wait(3000); // Wait for 2 seconds before retrying
        }
      }
      if (retryCount === 5) {
        console.error(`Unable to open URL after 5 retries: ${url}`);
        continue; // Skip to the next iteration if unable to open URL
      }

      // Rest of the code to extract data goes here

    }

    await page.close();
    await browser.close();

    console.log(laptopsData);

    const jsonData = JSON.stringify(laptopsData);
    fs.writeFileSync('laptops_data.json', jsonData);
    console.log('Data saved to laptops_data.json');
  } catch (error) {
    console.error('An error occurred:', error);
  }
}

// Call the function to start scraping
scrapePrices();


Promise { <pending> }


  Puppeteer old Headless deprecation warning:
    In the near feature `headless: true` will default to the new Headless mode
    for Chrome instead of the old Headless implementation. For more
    information, please see https://developer.chrome.com/articles/new-headless/.
    Consider opting in early by passing `headless: "new"` to `puppeteer.launch()`
    If you encounter any bugs, please report them to https://github.com/puppeteer/puppeteer/issues/new/choose.



Extracting links from: https://www.newegg.com/p/pl?N=100167751&page=1&LeftPriceRange=0+&recaptcha=pass
Found 36 links on page 1



  Puppeteer old Headless deprecation warning:
    In the near feature `headless: true` will default to the new Headless mode
    for Chrome instead of the old Headless implementation. For more
    information, please see https://developer.chrome.com/articles/new-headless/.
    Consider opting in early by passing `headless: "new"` to `puppeteer.launch()`
    If you encounter any bugs, please report them to https://github.com/puppeteer/puppeteer/issues/new/choose.



Extracting links from: https://www.newegg.com/p/pl?N=100167751&page=2&LeftPriceRange=0+&recaptcha=pass
Found 36 links on page 2


...


  Puppeteer old Headless deprecation warning:
    In the near feature `headless: true` will default to the new Headless mode
    for Chrome instead of the old Headless implementation. For more
    information, please see https://developer.chrome.com/articles/new-headless/.
    Consider opting in early by passing `headless: "new"` to `puppeteer.launch()`
    If you encounter any bugs, please report them to https://github.com/puppeteer/puppeteer/issues/new/choose.



Condition: 0
price: null
Failed to click the button or expected changes did not occur.
Value: NaN
Value: NaN
Value3: undefined
Value: NaN
Value: undefined
Value: NaN
Value: NaN
Value: 1
Value1: 1
Value2: 1
Value: NaN
Value: NaN
USB connections of different types: 0
Total USB connections: 0



  Puppeteer old Headless deprecation warning:
    In the near feature `headless: true` will default to the new Headless mode
    for Chrome instead of the old Headless implementation. For more
    information, please see https://developer.chrome.com/articles/new-headless/.
    Consider opting in early by passing `headless: "new"` to `puppeteer.launch()`
    If you encounter any bugs, please report them to https://github.com/puppeteer/puppeteer/issues/new/choose.



Condition: 0
price: null
Failed to click the button or expected changes did not occur.
Value: NaN
Value: NaN
Value3: undefined
Value: NaN
Value: undefined
Value: NaN
Value: NaN
Value: 1
Value1: 1
Value2: 1
Value: NaN
Value: NaN
USB connections of different types: 0
Total USB connections: 0



  Puppeteer old Headless deprecation warning:
    In the near feature `headless: true` will default to the new Headless mode
    for Chrome instead of the old Headless implementation. For more
    information, please see https://developer.chrome.com/articles/new-headless/.
    Consider opting in early by passing `headless: "new"` to `puppeteer.launch()`
    If you encounter any bugs, please report them to https://github.com/puppeteer/puppeteer/issues/new/choose.

TimeoutError: Navigation timeout of 30000 ms exceeded
    at LifecycleWatcher._LifecycleWatcher_createTimeoutPromise (C:\Users\nor29\node_modules\puppeteer\node_modules\puppeteer-core\lib\cjs\puppeteer\common\LifecycleWatcher.js:162:12)
An error occurred: TimeoutError: Navigation timeout of 30000 ms exceeded
    at LifecycleWatcher._LifecycleWatcher_createTimeoutPromise (C:\Users\nor29\node_modules\puppeteer\node_modules\puppeteer-core\lib\cjs\puppeteer\common\LifecycleWatcher.js:162:12)


In [7]:

const fileToLoad = 'C:\\Users\\nor29\\Desktop\\פלט תוכנית.txt';

fs.readFile(fileToLoad, 'utf8', (err, data) => {
  if (err) {
    console.error('Error reading the file:', err);
    return;
  }

  console.log('File contents:', data);
});


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

